In [1]:
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()
print("Cuda Available?  ", use_cuda)


Cuda Available?   True


In [2]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [3]:
import math

In [4]:
def ode_solver(z0, t0, t1, f):
    """
    Simplest Euler ODE Solver
    z0: value at intial state
    t0: intial state
    t1: final state to be calculated
    f: derivatve of a function with paramters z and p(aka parameter) with respect to t i.e f=dz/dt
    """
    h_max = .05 # Random Smallest Possible difference 
    n_steps = math.ceil((abs(t1-t0)/h_max).max().item()) # Based on distance between t1 and t0, number of steps to be taken with step length of h_max

    h = (t1 - t0)/n_steps # Approximated Smallest Possible Difference
    t = t0
    z = z0

    for _ in range(n_steps):
        z = z + h*f(z, t) # Euler Method: z_1 = z_0 + h*f(z, t)
        t = t + h # Updating t0 to reach t1 with step size of h
    
    # When we reach t1 from t0, its possible an ODE solver also makes the z0 to reach z1(the output)
    return z

In [ ]:
class BaseODESolver(nn.Module):
    """
    Base Class for Parameters based ODE Solver
    """

    def forward_with_grad(self, z, t, grad_ouptuts):
        """
        A custome method(not a method of nn.Module)
        Helps with calculation of coefficients required for reverse-mode automatic differentiation
        This will calculate the following:
        - Adjoint Sensitivity `a`
        - Vector field: df/dz
        - Rate of change of output with change in parameters: df/dp
        - 
        """